In [8]:
import numpy as np
import pandas as pd
import re
# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [287]:
data = pd.read_excel('BD_Technical_Team_Data_Challenge_Data copy.xlsx')

In [236]:
data.dtypes

Item_ID                      int64
Shopper_Entered_Price       object
Instacart_Current_Price    float64
Price_Per_Unit              object
Total_Orders                 int64
Flagged_Count                int64
dtype: object

In [343]:
#Item Ids with more than one entry
data[data['Item_ID'].isin(data['Item_ID'].value_counts()[data['Item_ID'].value_counts()>1].index)]

,Item_ID,Shopper_Entered_Price,Instacart_Current_Price,Price_Per_Unit,Total_Orders,Flagged_Count,Cleaned_Shopper_Entered_Price
3,926397,2.59,1.50,each,7,3,2.59
4,926397,2.69,1.50,each,7,1,2.69
10,962334,1.69,0.98,lb,23,2,1.69
11,962334,2,0.98,lb,23,2,2.00
17,1368891,1.29,0.79,each,15,1,1.29
18,1368891,1.3,0.79,each,15,3,1.30
19,1368891,1.28,0.79,each,15,4,1.28
49,2389320,3.49,1.99,lb,211,6,3.49
50,2389320,3 dollars and 49 cents,1.99,lb,211,1,3.49
51,2389320,349,1.99,lb,211,2,349.00


In [272]:
data['Item_ID'].value_counts()

5558931      5
9054234      3
1368891      3
131847318    3
7823079      3
3164058      3
2389320      3
108587358    2
4018347      2
7251867      2
926397       2
962334       2
8784279      2
3622608      2
4172760      2
107942175    2
91159371     2
132239592    2
6430212      2
90915336     2
132535827    2
8790354      2
2406033      1
2409606      1
195978879    1
132603048    1
107944614    1
7926912      1
104321871    1
20992986     1
            ..
9551718      1
19917009     1
964449       1
8975709      1
9110133      1
1974618      1
5494617      1
5558103      1
3401046      1
132101973    1
3165012      1
951120       1
4770639      1
2507598      1
8379288      1
90850122     1
2137338      1
3171654      1
5523165      1
4099905      1
131878206    1
3203901      1
9054522      1
8593209      1
1723833      1
6595893      1
90275634     1
3401001      1
6466131      1
131719680    1
Name: Item_ID, Length: 277, dtype: int64

In [289]:
#Dirty Data
data[data['Shopper_Entered_Price'].astype(str).str.contains('[A-Za-z]')]

,Item_ID,Shopper_Entered_Price,Instacart_Current_Price,Price_Per_Unit,Total_Orders,Flagged_Count
23,1712835,99 cents,0.50,each,732,1
50,2389320,3 dollars and 49 cents,1.99,lb,211,1
63,2520639,incorrect price,1.49,lb,339,1
72,3164058,the price is 625,3.50,each,128,3
86,3264804,new price is 4.39,2.59,each,19,2
99,4099905,price is different,1.29,lb,44,3
138,6430212,4 bucks,2.99,each,3,1
199,60790788,new price 429,2.50,each,13,1
225,91159371,price is 9.99,5.99,each,5,1
240,108587358,new price is 499,3.00,each,2,1


In [244]:
def validateString(s):
    letter_flag = False
    number_flag = False
    for i in s:
        if i.isalpha():
            letter_flag = True
        if i.isdigit():
            number_flag = True
    return letter_flag and number_flag

In [297]:
def clean_shopper_price(value):
    if validateString(value)==True:
        prices  = re.findall('(?P<value>[\d]*[.]?[\d]{1,2})\s*(?P<currency>cent|dollar)s?', value)
        final_price = 0.0
        if prices:
            for value, currency in prices:
                partial = float(value)
                if currency == 'cent':
                    final_price += partial / 100
                else:
                    final_price += partial 
        if not prices:
            prices= re.findall(r"[-+]?\d*\.\d+|\d+", value)
            for value in prices:
                final_price= value
    #elif all(i.isalpha() or i==' ' for i in value):
    else:
        final_price=value
        
    return final_price

In [291]:
data['Cleaned_Shopper_Entered_Price'] = data['Shopper_Entered_Price'].astype(str).apply(clean_shopper_price)
data['Cleaned_Shopper_Entered_Price'] = (data['Cleaned_Shopper_Entered_Price'].replace(',','.', regex=True)
                        .replace(' ','.', regex=True)
                        .astype(float))

In [318]:
data.loc[data.Cleaned_Shopper_Entered_Price == 'incorrect price', 'Cleaned_Shopper_Entered_Price'] = 1.49
data.loc[data.Cleaned_Shopper_Entered_Price == 'price is different', 'Cleaned_Shopper_Entered_Price'] = 1.29

In [323]:
data.dtypes

Item_ID                            int64
Shopper_Entered_Price             object
Instacart_Current_Price          float64
Price_Per_Unit                    object
Total_Orders                       int64
Flagged_Count                      int64
Cleaned_Shopper_Entered_Price    float64
dtype: object

In [329]:
wm = lambda x: np.average(x, weights=data.loc[x.index, "Flagged_Count"])
fun = {'Flagged_Count': ['sum'],'Total_Orders': ['sum'] , 'Instacart_Current_Price': ['mean'] ,\
    'Cleaned_Shopper_Entered_Price': {'weighted_mean' : wm}}

# Groupby aggregation:
data.groupby(["Item_ID"]).agg(fun)
#, "Flagged_Count", "Total_Orders", "Cleaned_Shopper_Entered_Price"

/anaconda3/lib/python3.7/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,Flagged_Count,Total_Orders,Instacart_Current_Price,Cleaned_Shopper_Entered_Price
,sum,sum,mean,weighted_mean
Item_ID,,,,
878229,1,14,1.50,2.590000
897786,1,29,1.50,2.590000
907794,6,44,7.98,13.790000
926397,4,14,1.50,2.615000
951120,1,2,2.98,5.790000
958545,2,20,2.59,360.000000
959931,3,18,1.99,3.590000
960372,3,146,0.25,0.500000


In [330]:
data2= data.groupby(["Item_ID"]).agg(f)
level0 = data2.columns.get_level_values(0)
data2.columns = level0
data2.head(500)
final=data2.reset_index().head(600)

/anaconda3/lib/python3.7/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [331]:
final

,Item_ID,Flagged_Count,Total_Orders,Instacart_Current_Price,Cleaned_Shopper_Entered_Price
0,878229,1,14,1.50,2.590000
1,897786,1,29,1.50,2.590000
2,907794,6,44,7.98,13.790000
3,926397,4,14,1.50,2.615000
4,951120,1,2,2.98,5.790000
5,958545,2,20,2.59,360.000000
6,959931,3,18,1.99,3.590000
7,960372,3,146,0.25,0.500000
8,960615,1,12,3.00,5.190000
9,962334,4,46,0.98,1.845000


In [340]:
def updated_price(final):
    updated_price= final['Cleaned_Shopper_Entered_Price']
    same_price= final['Instacart_Current_Price']
    if final['Flagged_Count']>1 and (final['Flagged_Count'] >= round(final['Total_Orders'] *.10)):
        return updated_price
    else:
        return same_price

In [341]:
final['New_Updated_price'] = final.apply(updated_price, axis=1)

In [342]:
final[50:]

,Item_ID,Flagged_Count,Total_Orders,Instacart_Current_Price,Cleaned_Shopper_Entered_Price,New_Updated_price
50,2450385,2,8,10.99,19.990000,19.99
51,2475126,1,25,4.99,7.990000,4.99
52,2488428,1,9,3.99,6.990000,3.99
53,2488437,1,8,3.99,6.990000,3.99
54,2491119,4,29,8.49,15.990000,15.99
55,2506140,1,4,5.99,9.990000,5.99
56,2507598,4,20,3.49,5.990000,5.99
57,2520639,1,339,1.49,1.490000,1.49
58,2986920,1,1,4.99,8.490000,4.99
59,2986938,1,3,4.99,8.490000,4.99
